In [16]:
import dash
import dash_core_components as dcc
import dash_html_components as html

# Define the Dash app
app = dash.Dash()

# Define the layout of the web page
app.layout = html.Div(
    html.Div([
        html.H1('Revive your pictures with a splash of colors!', 
                style={'color': 'white', 
                       'font-family': 'Lucida Handwriting',
                       'justify-content': 'center', 
                       'align-items': 'flex-start', 
                       'textAlign': 'center' }),
    ]),


    style={
        'background-image': 'url("/assets/splash.jpg")', 
        'background-size': 'cover',  
        'background-position': 'center',  
        'height': '100vh',
        'textAlign': 'center',  
        'display': 'flex',  
        'justifyContent': 'center',  
        'alignItems': 'center',  
    },
    
    
    
# add a font style to the text  
        # html.H1('Revive your pictures with a splash of colors!', 
        #         style={'color': 'white', 
        #                'font-family': 'Lucida Handwriting',
        #                'justify-content': 'center', 
        #                'align-items': 'flex-start', 
        #                'textAlign': 'center' }),

    
               
        html.H2('Image Upload', 
                        style={'color': 'red', 
                               'font-family': 'Papyrus', 
                               'text-align': 'left'}),
                    
            
        dcc.Upload(
            id='upload-image',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select an Image',style={'text-align': 'left'} )
            ]),
            style={
                'width': '50%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'margin': 'auto',
                'background-color': 'grey',
                'color': 'white', 
                'textAlign': 'center',
                'marginTop': '20px'
            },
            # Allow only images to be uploaded
            accept='image/*'
        ),
        html.Div(id='output-image-upload'),
    
)  


if __name__ == '__main__':
    app.run_server(port=4050)


SyntaxError: positional argument follows keyword argument (711270465.py, line 73)

In [46]:
import dash
import dash_core_components as dcc
import dash_html_components as html

# Define the Dash app
app = dash.Dash()

# Define the layout of the web page
app.layout = html.Div(
    children=[
        html.Div([
            html.H1('Revive your pictures with a splash of colors!', 
                    style={'color': 'red', 
                           'font-family': 'Lucida Handwriting',
                           'justify-content': 'center', 
                           'align-items': 'flex-start', 
                           'textAlign': 'center' }),
        ]),

        html.Div([
        html.Img(src="/assets/splash.jpg", 
                 style={
                        'background-size': 'cover', 
                        'width': '100%', 
                        'background-position': 'center',  
                        'height': '100vh',
                        'textAlign': 'center',  
                        'display': 'flex',  
                        'justifyContent': 'center',  
                        'alignItems': 'center'  }),

         html.Div(style={'position': 'absolute', 'top': '50%', 'left': '50%', 'transform': 'translate(-50%, -50%)'},
                 children=[
                     html.P(["Welcome to Colorify, where black and white memories",
                            html.Br(),
                            "find their vibrant voice! With our innovative models,",
                             html.Br(),
                            "we sprinkle a touch of magic onto your monochrome snapshots,",
                            html.Br(),
                            "revealing the hidden beauty of the past in vivid color. ",
                            html.Br(),
                            "Step into a world where nostalgia meets technicolor charm, ",
                            html.Br(),
                            "only at Colorify!", 
                     ],
                            style={'textAlign': 'left', 
                                   'color': 'white',
                                   'text-align': 'left',
                                   'transform': 'translate(-120%, -50%)'
                                   })
                 ])


    ]), 

        html.H2('Image Upload', 
                style={'color': 'red', 
                       'font-family': 'Papyrus', 
                       'text-align': 'left'}),
        dcc.Upload(
            id='upload-image',
            children=html.Div([
                'Drag and Drop or ',
                html.A('Select an Image',style={'text-align': 'left'} )
            ]),
            style={
                'width': '50%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'margin': 'auto',
                'background-color': 'grey',
                'color': 'white', 
                'textAlign': 'center',
                'marginTop': '20px'
            },
            # Allow only images to be uploaded
            accept='image/*'
        ),
        html.Div(id='output-image-upload'),
    ],
    
)



if __name__ == '__main__':
    app.run_server(port=4051)

In [21]:
import dash
import dash_core_components as dcc
import dash_html_components as html

# Define the Dash app
app = dash.Dash(__name__)

# Define the layout of the web page
app.layout = html.Div([
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select an Image')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow only images to be uploaded
        accept='image/*'
    ),
    html.Div(id='output-image-upload'),
])

if __name__ == '__main__':
    app.run_server(debug=True)
